<a href="https://colab.research.google.com/github/MarcosVeniciu/Producao-de-cafe-MG/blob/main/Coleta_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dados do IBGE sobre café

In [ ]:
!pip install geobr shapely geopandas pandas rasterio elevation

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
dic_mesorregioes = {
  "3101": "Noroeste de Minas/MG",
  "3102": "Norte de Minas/MG",
  "3103": "Jequitinhonha/MG",
  "3104": "Vale do Mucuri/MG",
  "3105": "Triângulo Mineiro/Alto Paranaíba/MG",
  "3106": "Central Mineira/MG",
  "3107": "Metropolitana de Belo Horizonte/MG",
  "3108": "Vale do Rio Doce/MG",
  "3109": "Oeste de Minas/MG",
  "3110": "Sul/Sudoeste de Minas/MG",
  "3111": "Campo das Vertentes/MG",
  "3112": "Zona da Mata/MG"
}

produtos = [
  "Café (em grão) Total",
  "Café (em grão) Arábica",
  "Café (em grão) Canephora"
]
variaveis = [
  "Mesorregião_id",
  "Mesorregião",
  "Municipio_id",
  "Área destinada à colheita (Hectares)",
  "Área colhida (Hectares)",
  "Quantidade produzida (Toneladas)",
  "Rendimento médio da produção (Quilogramas por Hectare)",
  "Valor da produção (Mil Reais)"
]

colunas= [
  "Mesorregião_id",
  "Mesorregião",
  "Municipio_id",
  "Municipio",
  "Produto",
  "Ano",
  "Área destinada à colheita (Hectares)",
  "Área colhida (Hectares)",
  "Quantidade produzida (Toneladas)",
  "Rendimento médio da produção (Quilogramas por Hectare)",
  "Valor da produção (Mil Reais)"
]

anos = list(range(1995, 2024))
lista_anos_chunks = [
  "1995",
  "1996|1997|1998|1999|2000",
  "2001|2002|2003|2004|2005",
  "2006|2007|2008|2009|2010",
  "2011|2012|2013|2014|2015",
  "2016|2017|2018|2019|2020",
  "2021|2022|2023"
]

# DataFrame final
df_geral = pd.DataFrame(columns=colunas)

for meso_id, meso_nome in dic_mesorregioes.items():
  print(f'Processando mesorregião {meso_nome} ({meso_id})…')
  # acumula todos os registros brutos para esta mesorregião
  dados = []
  municipios = set()

  for anos_str in lista_anos_chunks:
    url = (
      f"https://servicodados.ibge.gov.br/api/v3/agregados/1613"
      f"/periodos/{anos_str}/variaveis/2313|216|214|112|215"
      f"?localidades=N6[N8[{meso_id}]]&classificacao=82[2723,31619,31620]"
    )
    resp = requests.get(url)
    if resp.status_code != 200:
      print(f"  → chunk {anos_str} retornou {resp.status_code}, pulando…")
      continue

    result = resp.json()
    if not isinstance(result, list):
      print(f"  → resposta inesperada no chunk {anos_str}: {result}")
      continue

    # percorre as variáveis e séries desse pedaço
    for var in result:
      nome_var = f"{var['variavel']} ({var['unidade']})"
      for res in var["resultados"]:
        produto = list(res["classificacoes"][0]["categoria"].values())[0]
        for ser in res["series"]:
          muni = ser["localidade"]["nome"]
          muni_id = ser["localidade"]["id"]
          municipios.add((muni, muni_id))
          for ano in anos:
            # só adiciona se o ano estiver naquele chunk
            if str(ano) in anos_str.split("|"):
              dados.append({
                "Mesorregião_id": meso_id,
                "Mesorregião": meso_nome,
                "Municipio": muni,
                "Municipio_id": muni_id,
                "Produto": produto,
                "Ano": ano,
                nome_var: ser["serie"].get(str(ano))
              })

  # monta o DataFrame desta mesorregião
  df_result = pd.DataFrame(columns=colunas)
  idx = 0
  for muni, muni_id in sorted(municipios):
    for produto in produtos:
      for ano in anos:
        df_result.loc[idx, ["Mesorregião_id", "Mesorregião",
                            "Municipio", "Municipio_id",
                            "Produto", "Ano"]] = [
            meso_id, meso_nome, muni, muni_id, produto, ano
        ]
        # preenche as variáveis
        for var in variaveis:
          # busca no "dados" o registro correspondente
          for registro in dados:
            if (registro["Municipio"] == muni and
              registro["Produto"] == produto and
              registro["Ano"] == ano and
              var in registro):
              df_result.at[idx, var] = registro[var]
              break
        idx += 1

  # finalmente, concatena ao geral
  df_geral = pd.concat([df_geral, df_result], ignore_index=True)

print("Concluído. df_geral tem", len(df_geral), "linhas.")

Processando mesorregião Noroeste de Minas/MG (3101)…
Processando mesorregião Norte de Minas/MG (3102)…
Processando mesorregião Jequitinhonha/MG (3103)…
Processando mesorregião Vale do Mucuri/MG (3104)…
Processando mesorregião Triângulo Mineiro/Alto Paranaíba/MG (3105)…
Processando mesorregião Central Mineira/MG (3106)…
Processando mesorregião Metropolitana de Belo Horizonte/MG (3107)…
Processando mesorregião Vale do Rio Doce/MG (3108)…
Processando mesorregião Oeste de Minas/MG (3109)…
Processando mesorregião Sul/Sudoeste de Minas/MG (3110)…
Processando mesorregião Campo das Vertentes/MG (3111)…
Processando mesorregião Zona da Mata/MG (3112)…
Concluído. df_geral tem 76680 linhas.


In [ ]:
# 1) Se df_geral veio de um slice, faça uma cópia
df_mapeado = df_geral.copy()

# 2) Defina o mapeamento
mesorregioes_alvo = [
    'Campo Das Vertentes',
    'Central Mineira',
    'Jequitinhonha',
    'Metropolitana De Belo Horizonte',
    'Noroeste De Minas',
    'Norte De Minas',
    'Oeste De Minas',
    'Sul/Sudoeste De Minas',
    'Triângulo Mineiro/Alto Paranaíba',
    'Vale Do Mucuri',
    'Vale Do Rio Doce',
    'Zona Da Mata'
]

mesorregioes = [
    "Campo das Vertentes/MG",
    "Central Mineira/MG",
    "Jequitinhonha/MG",
    "Metropolitana de Belo Horizonte/MG",
    "Noroeste de Minas/MG",
    "Norte de Minas/MG",
    "Oeste de Minas/MG",
    "Sul/Sudoeste de Minas/MG",
    "Triângulo Mineiro/Alto Paranaíba/MG",
    "Vale do Mucuri/MG",
    "Vale do Rio Doce/MG",
    "Zona da Mata/MG"
]

mapeamento = dict(zip(mesorregioes, mesorregioes_alvo))

# 3) Aplique a substituição usando .loc
df_mapeado.loc[:, 'Mesorregião'] = df_mapeado['Mesorregião'].replace(mapeamento)

In [ ]:
# Os dados ausentes do dataset são preenchidos com '...', '-', então primeiro troco eles po <NA>.
df_replace = (
  df_mapeado
    .replace(['...', '-'], pd.NA)
    .infer_objects(copy=False)
)

In [ ]:
df_replace.to_excel("Indicadores IBGE por Municipio - 1995 a 2023.xlsx", index=False)

# Dados de longetude, latitude e altitude

## municipios

In [ ]:
import pandas as pd

In [ ]:
municipios = pd.read_excel("Indicadores IBGE por Municipio - 1995 a 2023.xlsx")
municipios.head(1)

,Mesorregião_id,Mesorregião,Municipio_id,Municipio,Produto,Ano,Área destinada à colheita (Hectares),Área colhida (Hectares),Quantidade produzida (Toneladas),Rendimento médio da produção (Quilogramas por Hectare),Valor da produção (Mil Reais)
0,3101,Noroeste De Minas,3104502,Arinos - MG,Café (em grão) Total,1995,NaN,NaN,NaN,NaN,NaN


In [ ]:
from shapely.geometry import shape
from tqdm import tqdm
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time

# --- Configuração de Session com retry/backoff ---
session = requests.Session()
retry_strategy = Retry(
    total=3,                    # até 3 tentativas
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"],
    backoff_factor=1,           # 1s, 2s, 4s entre tentativas
    raise_on_status=False
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)
session.mount("http://", adapter)

# Timeout padrão (connect, read)
TIMEOUT = (5, 10)

def obter_coordenadas(id_municipio, qualidade="intermediaria"):
    url = (
        f"https://servicodados.ibge.gov.br/api/v3/malhas/municipios/"
        f"{id_municipio}?formato=application/vnd.geo+json&qualidade={qualidade}"
    )
    for attempt in range(1, 4):
        try:
            resp = session.get(url, timeout=TIMEOUT)
            resp.raise_for_status()
            geojson = resp.json()
            geom = geojson["features"][0]["geometry"]
            poligono = shape(geom)
            centro = poligono.centroid
            return centro.y, centro.x  # (lat, lon)
        except requests.RequestException as e:
            print(f"[tentativa {attempt}] erro ao obter coords do {id_municipio}: {e}")
            if attempt < 3:
                time.sleep(2 ** (attempt - 1))  # backoff exponencial
            else:
                raise

def obter_altitude(lat, lon):
    url = "https://api.open-meteo.com/v1/elevation"
    params = {"latitude": lat, "longitude": lon}
    for attempt in range(1, 4):
        try:
            resp = session.get(url, params=params, timeout=TIMEOUT)
            resp.raise_for_status()
            data = resp.json()
            return data["elevation"][0]
        except requests.RequestException as e:
            print(f"[tentativa {attempt}] erro ao obter altitude ({lat},{lon}): {e}")
            if attempt < 3:
                time.sleep(2 ** (attempt - 1))
            else:
                return None  # ou lance: raise

if __name__ == "__main__":
  municipios_dict = {}
  for _, row in municipios.iterrows():
      mid = row['Municipio_id']
      if mid not in municipios_dict:
          municipios_dict[mid] = {
              'Mesorregião_id': row['Mesorregião_id'],
              'id': mid,
              'nome': row['Municipio']
          }

  registros = []
  for info in tqdm(municipios_dict.values(), desc="Processando"):
      try:
          lat, lon = obter_coordenadas(info["id"])
          alt = obter_altitude(lat, lon)
      except Exception as e:
          print(f"Pulando município {info['id']} devido a erro persistente: {e}")
          continue

      registros.append({
          "Mesorregião_id": info["Mesorregião_id"],
          "municipio_id":    info["id"],
          "municipio_nome":  info["nome"],
          "lat":             round(lat, 6),
          "lon":             round(lon, 6),
          "alt":             round(alt, 1) if alt is not None else None
      })

  df = pd.DataFrame(registros)
  df.to_excel(
      "municipios_georeferenciamento.xlsx",
      sheet_name="Municípios",
      index=False
  )
  print("\nPlanilha gerada: municipios_georeferenciamento.xlsx")

Processando: 100%|██████████| 852/852 [03:31<00:00,  4.02it/s]


Planilha gerada: municipios_georeferenciamento.xlsx


# Dados climaticos


## Municipios

https://worldclim.org/data/monthlywth.html

In [ ]:
!pip install rasterio xarray pandas requests tqdm

In [2]:
import os
import re
import time
import zipfile
from io import BytesIO

import pandas as pd
import rasterio
import requests
from shapely.geometry import shape
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# --- CONFIGURAÇÕES GLOBAIS ---

# Timeouts para todas as requisições HTTP: (connect_timeout, read_timeout)
TIMEOUT = (5, 15)

# Serviço de malhas do IBGE e de altitude
IBGE_URL = "https://servicodados.ibge.gov.br/api/v3/malhas/municipios/{id}"
ELEV_URL = "https://api.open-meteo.com/v1/elevation"

# WorldClim 2.1 histórica CRU-TS 4.06 2.5'
WC_BASE_URL = "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09"
VARIAVEIS = ["prec", "tmin", "tmax"]
DECADAS   = ["1990-1999", "2000-2009", "2010-2019", "2020-2024"]
WC_DIR    = "wc_monthly"

# Regex para extrair ano e mês dos arquivos .tif
TIF_RE = re.compile(r".*_(\d{4})-(\d{2})\.tif$")

# --- SESSION COM RETRY / BACKOFF PARA O SERVIÇO IBGE ---

session = requests.Session()
retry_strategy = Retry(
    total=3,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)
session.mount("http://", adapter)

# --- FUNÇÕES AUXILIARES ---

def obter_coordenadas(id_municipio, qualidade="intermediaria"):
    """
    Retorna (lat, lon) do centróide do município IBGE.
    Faz até 3 tentativas em falha de rede.
    """
    url = IBGE_URL.format(id=id_municipio) + f"?formato=application/vnd.geo+json&qualidade={qualidade}"
    for attempt in range(1, 4):
        try:
            r = session.get(url, timeout=TIMEOUT)
            r.raise_for_status()
            geo = r.json()
            geom = geo["features"][0]["geometry"]
            poly = shape(geom)
            c = poly.centroid
            return c.y, c.x
        except Exception as e:
            print(f"[coords][{id_municipio}] tentativa {attempt} falhou: {e}")
            if attempt < 3:
                time.sleep(2 ** (attempt - 1))
            else:
                raise

def baixar_e_extrair_wc():
    """
    Baixa e extrai os ZIPs do WorldClim para cada variável e década.
    Pula se o diretório de destino já existir.
    """
    os.makedirs(WC_DIR, exist_ok=True)
    for dec in DECADAS:
        for var in VARIAVEIS:
            dest = os.path.join(WC_DIR, f"{var}_{dec}")
            if not os.path.isdir(dest):
                zip_fn = f"wc2.1_cruts4.09_2.5m_{var}_{dec}.zip"
                print(f"Baixando {zip_fn} …")
                url = f"{WC_BASE_URL}/{zip_fn}"
                r = requests.get(url, timeout=TIMEOUT)
                r.raise_for_status()
                with zipfile.ZipFile(BytesIO(r.content)) as z:
                    z.extractall(dest)

def sample_clima(var, dec, lat, lon):
    """
    Amostra todos os GeoTIFFs em WC_DIR/var_dec para o ponto (lat, lon).
    Retorna DataFrame com colunas [Ano, Mês, Valor].
    """
    folder = os.path.join(WC_DIR, f"{var}_{dec}")
    recs = []
    for fn in os.listdir(folder):
        m = TIF_RE.match(fn)
        if not m:
            continue
        ano, mes = int(m.group(1)), int(m.group(2))
        path = os.path.join(folder, fn)
        with rasterio.open(path) as src:
            val = next(src.sample([(lon, lat)]))[0]
        recs.append({"Ano": ano, "Mês": mes, "Valor": val})
    return pd.DataFrame(recs)

# --- SCRIPT PRINCIPAL ---

if __name__ == "__main__":
  df = pd.read_excel("Indicadores IBGE por Municipio - 1995 a 2023.xlsx")

  # 2) Monta dicionário único de municípios
  municipios = {}
  for _, row in df.iterrows():
    mid = row["Municipio_id"]
    if mid not in municipios:
      municipios[mid] = {
          "Mesorregião_id": row["Mesorregião_id"],
          "municipio_id":   mid,
          "municipio_nome": row["Municipio"]
      }

  # 3) Baixa e extrai os dados do WorldClim, se necessário
  baixar_e_extrair_wc()

  # 4) Para cada município, coleta coords e amostra clima
  todos = []
  for info in tqdm(municipios.values(), desc="Municípios"):
    try:
      lat, lon = obter_coordenadas(info["municipio_id"])
    except Exception:
      print(f"  → pulo {info['municipio_id']}: coords indisponíveis")
      continue

    for dec in DECADAS:
      for var in VARIAVEIS:
        df_s = sample_clima(var, dec, lat, lon)
        df_s["Mesorregião_id"] = info["Mesorregião_id"]
        df_s["municipio_id"]   = info["municipio_id"]
        df_s["municipio_nome"] = info["municipio_nome"]
        df_s["Variável"]       = var
        df_s["Década"]         = dec
        todos.append(df_s)

  # 5) Consolida e pivot para formato wide
  df_clim = pd.concat(todos, ignore_index=True)
  df_wide = df_clim.pivot_table(
      index=["Mesorregião_id","municipio_id","municipio_nome","Ano","Mês"],
      columns="Variável",
      values="Valor"
  ).reset_index().rename(columns={
      "prec": "prec_mm",
      "tmin": "tmin_°C",
      "tmax": "tmax_°C"
  }).sort_values(["municipio_id","Ano","Mês"]).reset_index(drop=True)

  # 6) Salva em CSV
  out_fn = "climate_por_municipio_1990_2024.csv"
  df_wide.to_csv(out_fn, index=False, encoding="utf-8-sig")
  print(f"→ Arquivo gerado: {out_fn}")

Baixando wc2.1_cruts4.09_2.5m_prec_1990-1999.zip …
Baixando wc2.1_cruts4.09_2.5m_tmin_1990-1999.zip …
Baixando wc2.1_cruts4.09_2.5m_tmax_1990-1999.zip …
Baixando wc2.1_cruts4.09_2.5m_prec_2000-2009.zip …
Baixando wc2.1_cruts4.09_2.5m_tmin_2000-2009.zip …
Baixando wc2.1_cruts4.09_2.5m_tmax_2000-2009.zip …
Baixando wc2.1_cruts4.09_2.5m_prec_2010-2019.zip …
Baixando wc2.1_cruts4.09_2.5m_tmin_2010-2019.zip …
Baixando wc2.1_cruts4.09_2.5m_tmax_2010-2019.zip …
Baixando wc2.1_cruts4.09_2.5m_prec_2020-2024.zip …
Baixando wc2.1_cruts4.09_2.5m_tmin_2020-2024.zip …
Baixando wc2.1_cruts4.09_2.5m_tmax_2020-2024.zip …


Municípios: 100%|██████████| 852/852 [31:06<00:00,  2.19s/it]


→ Arquivo gerado: climate_por_municipio_1990_2024.csv
